In [1]:
import gc
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from bayes_opt import BayesianOptimization

In [16]:
def xgb_bo(max_depth, min_child_weight, subsample, colsample_bytree):
    
    t = time.time()
    params = {
    'seed': 1337,
    'colsample_bytree': max(min(colsample_bytree, 1), 0),
    'silent': 1,
    'subsample': max(min(subsample, 1), 0),
    'eta': 0.05,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': int(max_depth),
    'min_child_weight': int(min_child_weight),
    'nthread': 4,
    'tree_method': 'hist',
    }
    
    dtrain = xgb.DMatrix(X_tr, label = y_tr)
    dval = xgb.DMatrix(X_val, label = y_val)
    watchlist = [(dtrain, 'train'), (dval, 'valid')]

    print('Start training...')
    gbm = xgb.train(params, dtrain, 100000, watchlist, 
                    early_stopping_rounds = 100, verbose_eval = 100)
    val_pred = gbm.predict(xgb.DMatrix(X_val), ntree_limit=gbm.best_ntree_limit)
    score = log_loss(y_val, val_pred)
    print('Final score:', score, '\n', 'Time it took to train and predict:', time.time() - t)
    return -score


def lgb_bo(max_depth, min_data_in_leaf, subsample, colsample_bytree, feature_fraction):
    params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'binary',
        'metric' : {'binary_logloss'},
        'learning_rate' : 0.05,
        'feature_fraction' : max(min(feature_fraction, 1), 0),
        'bagging_fraction': 0.9,
        'bagging_freq': 100,
        'num_leaves' : 255,
        'max_depth': int(max_depth),
        'min_data_in_leaf': int(min_data_in_leaf),
        'subsample': max(min(subsample, 1), 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'silent': 1,
        'random_state': 1337,
        'verbose': 1,
        'nthread': 9,
    }

    lgb_train = lgb.Dataset(X_tr, y_tr.is_duplicate.values)
    lgb_val = lgb.Dataset(X_val, y_val.is_duplicate.values)
    t = time.time()
    print('Start training...')
    gbm = lgb.train(params, lgb_train, num_boost_round = 100000, valid_sets = lgb_val,
                    early_stopping_rounds = 100, verbose_eval = 100)
    val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
    score = log_loss(y_val, val_pred)
    print('Final score:', score, '\n', 'Time it took to train and predict:', time.time() - t)
    return -score

In [6]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
feats_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/uncleaned/'
trans_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/lemmatized_fullclean/transformations/'

#X_train = pd.read_pickle('Xtrain_814colsBest.pkl', compression = 'bz2')
xgb_feats = pd.read_csv(feats_src + '/the_1owl/owl_train.csv')
y_train = xgb_feats[['is_duplicate']]

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify = y_train,
                                                    test_size = 0.2, random_state = 111)

del xgb_feats, X_train
gc.collect()

1547

 Step |   Time |      Value |   colsample_bytree |   feature_fraction |   max_depth |   min_data_in_leaf |   subsample | 

  20 | 05m16s |   -0.20453 |             0.4089 |             0.5045 |      8.0778 |            23.0794 |      0.6507 
  
   23 | 06m22s |   -0.20448 |             0.4017 |             0.5290 |      8.0352 |            27.5745 |      0.8080 | 
   
  19 | 05m29s |   -0.20479 |             0.6450 |             0.5302 |      8.0956 |            21.4192 |      0.8488 | 

In [15]:
lgbBO = BayesianOptimization(lgb_bo, {
        'max_depth': (8, 15),
        'min_data_in_leaf': (8, 28),
        'subsample': (0.65, 0.85),
        'colsample_bytree': (0.4, 0.65),
        'feature_fraction': (0.5, 0.95),
    })

num_iter = 15
init_points = 10
lgbBO.maximize(init_points=init_points, n_iter=num_iter)
print('lgb: %f' % lgbBO.res['max']['max_val'])

Initialization
------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   feature_fraction |   max_depth |   min_data_in_leaf |   subsample | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.229931
[200]	valid_0's binary_logloss: 0.216166
[300]	valid_0's binary_logloss: 0.21122
[400]	valid_0's binary_logloss: 0.208599
[500]	valid_0's binary_logloss: 0.207171
[600]	valid_0's binary_logloss: 0.206397
[700]	valid_0's binary_logloss: 0.205852
[800]	valid_0's binary_logloss: 0.205352
[900]	valid_0's binary_logloss: 0.205371
Early stopping, best iteration is:
[865]	valid_0's binary_logloss: 0.205265
Final score: 0.205276358316 
 Time it took to train and predict: 373.49435925483704
    1 | 06m13s |   -0.20528 |             0.4130 |             0.9248 |      8.9347 |            21.2268 |      0.7029 | 
Start training...
T

   12 | 06m47s |   -0.20499 |             0.6318 |             0.8781 |      8.0205 |            27.9379 |      0.6543 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.230284
[200]	valid_0's binary_logloss: 0.216782
[300]	valid_0's binary_logloss: 0.211977
[400]	valid_0's binary_logloss: 0.209541
[500]	valid_0's binary_logloss: 0.207918
[600]	valid_0's binary_logloss: 0.206994
[700]	valid_0's binary_logloss: 0.206479
[800]	valid_0's binary_logloss: 0.206049
[900]	valid_0's binary_logloss: 0.205952
[1000]	valid_0's binary_logloss: 0.205934
Early stopping, best iteration is:
[947]	valid_0's binary_logloss: 0.205746
Final score: 0.205752076008 
 Time it took to train and predict: 388.84168434143066


/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   13 | 06m52s |   -0.20575 |             0.6447 |             0.8808 |      8.0796 |            12.3303 |      0.6784 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.220779
[200]	valid_0's binary_logloss: 0.210712
[300]	valid_0's binary_logloss: 0.207313
[400]	valid_0's binary_logloss: 0.206145
[500]	valid_0's binary_logloss: 0.206329
Early stopping, best iteration is:
[407]	valid_0's binary_logloss: 0.206071
Final score: 0.206078519844 
 Time it took to train and predict: 438.6605553627014


/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.29988774e-05]), 'warnflag': 2, 'funcalls': 49, 'nit': 5, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


...

   18 | 07m09s |   -0.20560 |             0.4284 |             0.9248 |      8.0646 |            15.9829 |      0.8443 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.229888
[200]	valid_0's binary_logloss: 0.216594
[300]	valid_0's binary_logloss: 0.211609
[400]	valid_0's binary_logloss: 0.209338
[500]	valid_0's binary_logloss: 0.207726
[600]	valid_0's binary_logloss: 0.206492
[700]	valid_0's binary_logloss: 0.205664
[800]	valid_0's binary_logloss: 0.205135
[900]	valid_0's binary_logloss: 0.20493
[1000]	valid_0's binary_logloss: 0.204946
Early stopping, best iteration is:
[923]	valid_0's binary_logloss: 0.204787
Final score: 0.204794163891 
 Time it took to train and predict: 299.51048970222473
   19 | 05m29s |   -0.20479 |             0.6450 |             0.5302 |      8.0956 |            21.4192 |      0.8488 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.230152

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00021708]), 'warnflag': 2, 'funcalls': 47, 'nit': 3, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


   21 | 05m32s |   -0.20505 |             0.4133 |             0.5195 |      8.0410 |            10.2447 |      0.8379 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.224017
[200]	valid_0's binary_logloss: 0.212515
[300]	valid_0's binary_logloss: 0.208185
[400]	valid_0's binary_logloss: 0.20654
[500]	valid_0's binary_logloss: 0.205621
[600]	valid_0's binary_logloss: 0.205788
Early stopping, best iteration is:
[539]	valid_0's binary_logloss: 0.205497
Final score: 0.205511489123 
 Time it took to train and predict: 262.5262417793274
   22 | 04m55s |   -0.20551 |             0.4192 |             0.5076 |     10.8824 |            12.4620 |      0.6522 | 
Start training...
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's binary_logloss: 0.230374
[200]	valid_0's binary_logloss: 0.216548
[300]	valid_0's binary_logloss: 0.21135
[400]	valid_0's binary_logloss: 0.208854
[500]	valid_0's binary_logloss: 0.206996
[6

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.63786060e-05]), 'warnflag': 2, 'funcalls': 50, 'nit': 6, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)
/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -7.46082515e-05]), 'warnflag': 2, 'funcalls': 53, 'nit': 5, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


   25 | 06m34s |   -0.20523 |             0.4407 |             0.5080 |     14.9286 |            12.5507 |      0.6762 | 
lgb: -0.204484


 Step |   Time |      Value |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 

* 13 | 37m14s |   -0.20249 |             0.4124 |      8.1496 |            17.5251 |      0.8456 | 
  
* 15 | 27m07s |   -0.20227 |             0.4243 |      8.4558 |            19.5977 |      0.8486 | 

* 2 | 26m23s |   -0.20295 |             0.5589 |     10.5015 |            21.8423 |      0.8243 | 

In [ ]:
xgbBO = BayesianOptimization(xgb_bo, {
        'max_depth': (8, 15),
        'min_child_weight': (8, 28),
        'subsample': (0.65, 0.85),
        'colsample_bytree': (0.4, 0.65),
    })

num_iter = 15
init_points = 10
xgbBO.maximize(init_points=init_points, n_iter=num_iter)
print('XGB: %f' % xgbBO.res['max']['max_val'])

Initialization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 
Start training...
[0]	train-logloss:0.661081	valid-logloss:0.661342
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.21886	valid-logloss:0.23045
[200]	train-logloss:0.192722	valid-logloss:0.217202
[300]	train-logloss:0.176578	valid-logloss:0.212179
[400]	train-logloss:0.163423	valid-logloss:0.209578
[500]	train-logloss:0.15271	valid-logloss:0.207983
[600]	train-logloss:0.143403	valid-logloss:0.2069
[700]	train-logloss:0.135151	valid-logloss:0.206017
[800]	train-logloss:0.127028	valid-logloss:0.205272
[900]	train-logloss:0.119983	valid-logloss:0.204507
[1000]	train-logloss:0.113577	valid-logloss:0.204169
[1100]	train-logloss:0.107496	valid-logloss:0.20

[0]	train-logloss:0.659927	valid-logloss:0.660615
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.186458	valid-logloss:0.221379
[200]	train-logloss:0.147287	valid-logloss:0.210854
[300]	train-logloss:0.122315	valid-logloss:0.207414
[400]	train-logloss:0.103355	valid-logloss:0.205813
[500]	train-logloss:0.088769	valid-logloss:0.20486
[600]	train-logloss:0.077031	valid-logloss:0.204677
[700]	train-logloss:0.067276	valid-logloss:0.204414
[800]	train-logloss:0.058638	valid-logloss:0.204595
Stopping. Best iteration:
[726]	train-logloss:0.064943	valid-logloss:0.204322

Final score: 0.204322113779 
 Time it took to train and predict: 1450.4375236034393
    8 | 24m10s |   -0.20432 |             0.4145 |     12.8761 |            16.7429 |      0.7786 | 
Start training...
[0]	train-logloss:0.660308	valid-logloss:0.660758
Multiple eval metrics have been passed: 'valid-logl

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00021962]), 'warnflag': 2, 'funcalls': 52, 'nit': 4, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)
/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 
Start training...
[0]	train-logloss:0.659843	valid-logloss:0.660573
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.18088	valid-logloss:0.220866
[200]	train-logloss:0.144119	valid-logloss:0.210647
[300]	train-logloss:0.120296	valid-logloss:0.207447
[400]	train-logloss:0.102178	valid-logloss:0.205689
[500]	train-logloss:0.087944	valid-logloss:0.204855
[600]	train-logloss:0.07673	valid-logloss:0.204424
[700]	train-logloss:0.067008	valid-logloss:0.204355
Stopping. Best iteration:
[656]	train-logloss:0.070956	valid-logloss:0.204323

Final score: 0.204322676716 
 Time it took to train and predict: 1420.6970834732056


/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0001811]), 'warnflag': 2, 'funcalls': 55, 'nit': 5, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)
/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -5.32737031e-05]), 'warnflag': 2, 'funcalls': 48, 'nit': 4, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


...

   14 | 20m04s |   -0.20647 |             0.4110 |     14.7764 |             8.0311 |      0.8391 | 
Start training...
[0]	train-logloss:0.66107	valid-logloss:0.661323
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.218422	valid-logloss:0.230321
[200]	train-logloss:0.191995	valid-logloss:0.217002
[300]	train-logloss:0.175714	valid-logloss:0.212014
[400]	train-logloss:0.162301	valid-logloss:0.209233
[500]	train-logloss:0.152167	valid-logloss:0.207414
[600]	train-logloss:0.142129	valid-logloss:0.206144
[700]	train-logloss:0.133907	valid-logloss:0.205265
[800]	train-logloss:0.126027	valid-logloss:0.204575
[900]	train-logloss:0.118711	valid-logloss:0.203844
[1000]	train-logloss:0.111884	valid-logloss:0.203376
[1100]	train-logloss:0.105505	valid-logloss:0.203004
[1200]	train-logloss:0.099742	valid-logloss:0.202735
[1300]	train-logloss:0.094174	valid-logloss:0.20253
[

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00183407]), 'warnflag': 2, 'funcalls': 66, 'nit': 6, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


   15 | 27m07s |   -0.20227 |             0.4243 |      8.4558 |            19.5977 |      0.8486 | 
Start training...
[0]	train-logloss:0.660707	valid-logloss:0.661036
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.211434	valid-logloss:0.22772
[200]	train-logloss:0.182043	valid-logloss:0.215035
[300]	train-logloss:0.162723	valid-logloss:0.210134
[400]	train-logloss:0.148663	valid-logloss:0.207879
[500]	train-logloss:0.136971	valid-logloss:0.206427
[600]	train-logloss:0.126185	valid-logloss:0.205165
[700]	train-logloss:0.116983	valid-logloss:0.20446
[800]	train-logloss:0.108245	valid-logloss:0.203843
[900]	train-logloss:0.100276	valid-logloss:0.203442
[1000]	train-logloss:0.093177	valid-logloss:0.202941
[1100]	train-logloss:0.08684	valid-logloss:0.202684
[1200]	train-logloss:0.081207	valid-logloss:0.20269
[1300]	train-logloss:0.076136	valid-logloss:0.20271
Stop

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.01960263e-05]), 'warnflag': 2, 'funcalls': 58, 'nit': 8, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH'}
  " state: %s" % convergence_dict)


   16 | 25m08s |   -0.20265 |             0.4001 |      9.4882 |            20.5449 |      0.8479 | 
Start training...
[0]	train-logloss:0.660908	valid-logloss:0.66118
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.218937	valid-logloss:0.230349
[200]	train-logloss:0.192589	valid-logloss:0.216864
[300]	train-logloss:0.175922	valid-logloss:0.211887
[400]	train-logloss:0.162866	valid-logloss:0.20914
[500]	train-logloss:0.152061	valid-logloss:0.207336
[600]	train-logloss:0.142926	valid-logloss:0.20624
[700]	train-logloss:0.134463	valid-logloss:0.205354
[800]	train-logloss:0.126287	valid-logloss:0.20473
[900]	train-logloss:0.118922	valid-logloss:0.204164
